In [ ]:
pip install opencv-python

In [ ]:
import os
import shutil
import glob
import cv2
import signal
import sys
from concurrent.futures import ThreadPoolExecutor, as_completed

DATA_ROOT = "/Data/aa/nutritionALL/imagery/side_angles"
OUTPUT_ROOT = "/Data/aa/nutritionALL/imagery/extracted_frames_jpg"
os.makedirs(OUTPUT_ROOT, exist_ok=True)

FRAMES_PER_CAMERA = 2
TARGET_WIDTH = 640  
EXPECTED_PER_CAMERA = FRAMES_PER_CAMERA  

for dish_name in os.listdir(OUTPUT_ROOT):
    dish_path = os.path.join(OUTPUT_ROOT, dish_name)
    if not os.path.isdir(dish_path):
        continue
    all_cameras_good = True
    for cam_folder in os.listdir(dish_path):
        cam_path = os.path.join(dish_path, cam_folder)
        if not os.path.isdir(cam_path):
            continue
        jpgs = [f for f in os.listdir(cam_path) if f.lower().endswith(".jpg")]
        if len(jpgs) < EXPECTED_PER_CAMERA:
            all_cameras_good = False
            break
    if not all_cameras_good:
        print(f"Deleting incomplete dish folder: {dish_path}")
        shutil.rmtree(dish_path)

In [ ]:
import os
import cv2
import signal

should_stop = False

def signal_handler(sig, frame):
    global should_stop
    print("Interrupt received! Stopping new tasks…")
    should_stop = True

signal.signal(signal.SIGINT, signal_handler)

def extract_two_frames_from_video(video_path, out_dir):
    global should_stop
    try:
        if should_stop:
            return "STOPPED (user interrupt)"
        if os.path.isdir(out_dir):
            existing_jpgs = [f for f in os.listdir(out_dir) if f.lower().endswith(".jpg")]
            if len(existing_jpgs) == FRAMES_PER_CAMERA:
                return f"SKIPPED (already has {FRAMES_PER_CAMERA} JPGs): {os.path.relpath(video_path, DATA_ROOT)}"
        else:
            os.makedirs(out_dir, exist_ok=True)

        basename = os.path.splitext(os.path.basename(video_path))[0] 
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            return f"SKIPPED (cannot open): {os.path.relpath(video_path, DATA_ROOT)}"

        saved_count = 0
        total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
        if total > FRAMES_PER_CAMERA:
            idx1 = total // 3
            idx2 = (2 * total) // 3
            for i, idx in enumerate([idx1, idx2], start=1):
                if should_stop:
                    cap.release()
                    return "STOPPED (user interrupt)"
                cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
                ret, frame = cap.read()
                if not ret or frame is None:
                    continue
                h, w = frame.shape[:2]
                new_h = int(h * TARGET_WIDTH / w)
                resized = cv2.resize(frame, (TARGET_WIDTH, new_h), interpolation=cv2.INTER_AREA)
                fname = f"{basename}_frame_{i}_of_{FRAMES_PER_CAMERA}.jpg"
                out_path = os.path.join(out_dir, fname)
                success = cv2.imwrite(out_path, resized, [cv2.IMWRITE_JPEG_QUALITY, 90])
                if success and os.path.exists(out_path) and os.path.getsize(out_path) > 0:
                    saved_count += 1
            cap.release()

        if saved_count < FRAMES_PER_CAMERA:
            cap = cv2.VideoCapture(video_path)
            if not cap.isOpened():
                return f"SKIPPED (cannot reopen): {os.path.relpath(video_path, DATA_ROOT)}"
            frames = []
            while True:
                if should_stop:
                    cap.release()
                    return "STOPPED (user interrupt)"
                ret, f = cap.read()
                if not ret:
                    break
                frames.append(f)
            cap.release()
            N = len(frames)
            if N < FRAMES_PER_CAMERA:
                return f"SKIPPED (only {N} frames): {os.path.relpath(video_path, DATA_ROOT)}"
            saved_count = 0
            # Optionally, remove any partial files before fallback:
            for f in os.listdir(out_dir):
                if f.lower().endswith(".jpg"):
                    os.remove(os.path.join(out_dir, f))
            idx1 = N // 3
            idx2 = (2 * N) // 3
            for i, idx in enumerate([idx1, idx2], start=1):
                if should_stop:
                    return "STOPPED (user interrupt)"
                frame = frames[idx]
                h, w = frame.shape[:2]
                new_h = int(h * TARGET_WIDTH / w)
                resized = cv2.resize(frame, (TARGET_WIDTH, new_h), interpolation=cv2.INTER_AREA)
                fname = f"{basename}_frame_{i}_of_{FRAMES_PER_CAMERA}.jpg"
                out_path = os.path.join(out_dir, fname)
                success = cv2.imwrite(out_path, resized, [cv2.IMWRITE_JPEG_QUALITY, 90])
                if success and os.path.exists(out_path) and os.path.getsize(out_path) > 0:
                    saved_count += 1
        return f"SAVED {saved_count}/{FRAMES_PER_CAMERA} JPGs: {os.path.relpath(video_path, DATA_ROOT)}"
    except Exception as e:
        return f"ERROR processing {os.path.relpath(video_path, DATA_ROOT)}: {e}"

In [ ]:
def main():
    global should_stop
    video_paths = glob.glob(os.path.join(DATA_ROOT, "**", "*.h264"), recursive=True)
    print(f"Found {len(video_paths)} .h264 videos to process.\n")
    if not video_paths:
        print("No .h264 files found under DATA_ROOT. Check your path.")
        return
    test_video = video_paths[0]
    test_out = os.path.join(OUTPUT_ROOT, "test_dish", "camera_test_frames")
    print("🧪 Testing on first video:", test_video)
    test_status = extract_two_frames_from_video(test_video, test_out)
    print("   Test result:", test_status)
    if test_status.startswith("ERROR") or test_status.startswith("SKIPPED (cannot open") or test_status.startswith("SKIPPED (only"):
        print("Test indicates a problem.")
        return
    #NUM_WORKERS = min(4, os.cpu_count() or 1)
    NUM_WORKERS = 12
    print(f"Processing with {NUM_WORKERS} workers\n")
    processed_count = 0
    skipped_count = 0
    error_count = 0
    try:
        with ThreadPoolExecutor(max_workers=NUM_WORKERS) as executor:
            future_to_vid = {}
            for vid in video_paths:
                if should_stop:
                    break
                rel = os.path.relpath(vid, DATA_ROOT) # e.g. "dish_12345/camera_A.h264"
                dish = os.path.dirname(rel) # e.g. "dish_12345"
                cam  = os.path.splitext(os.path.basename(vid))[0] # e.g. "camera_A"
                out_dir = os.path.join(OUTPUT_ROOT, dish, cam + "_frames")
                future = executor.submit(extract_two_frames_from_video, vid, out_dir)
                future_to_vid[future] = vid

            for future in as_completed(future_to_vid):
                if should_stop:
                    break
                vid = future_to_vid[future]
                try:
                    status = future.result(timeout=120)
                except Exception as e:
                    status = f"ERROR: {os.path.relpath(vid, DATA_ROOT)} – {e}"
                print(status)
                if status.startswith("SAVED"):
                    processed_count += 1
                elif status.startswith("SKIPPED"):
                    skipped_count += 1
                else:
                    error_count += 1

    except KeyboardInterrupt:
        print("\n Detected user interrupt. Exiting early")
        should_stop = True

    # Summary
    print("\n Summary:")
    print(f"Processed: {processed_count}")
    print(f"Skipped: {skipped_count}")
    print(f"Errors: {error_count}")
    print(f"Total: {len(video_paths)}")

    # Final cleanup: delete any dish folder with incomplete cameras
    print("\n Performing final cleanup of incomplete dishes")
    for dish_name in os.listdir(OUTPUT_ROOT):
        dish_path = os.path.join(OUTPUT_ROOT, dish_name)
        if not os.path.isdir(dish_path):
            continue
        all_good = True
        for cam_folder in os.listdir(dish_path):
            cam_path = os.path.join(dish_path, cam_folder)
            if not os.path.isdir(cam_path):
                continue
            jpgs = [f for f in os.listdir(cam_path) if f.lower().endswith(".jpg")]
            if len(jpgs) < FRAMES_PER_CAMERA:
                all_good = False
                break
        if not all_good:
            print(f"Deleting incomplete dish folder: {dish_path}")
            shutil.rmtree(dish_path)
    print("Final cleanup complete. Only fully complete dishes remain.")

if __name__ == "__main__":
    main()
